In [18]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys, io

In [19]:
path = get_file(
    'nietzsche.txt', 
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
# need to get a source of the content

with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

print('text corpus length: ', len(text))

text corpus length:  600893


In [20]:
chars = sorted(list(set(text)))
numChars = len(chars)
print() # print total number of chars

char_indices = { ch : i for i, ch in enumerate(chars)}
indices_char = { i : ch for i, ch in enumerate(chars)}
# indices_char2 = { i : ch for ch, i in char_indices.items()}
# print(char_indices)
# print(indices_char1)

In [21]:
def create_seq(text, maxLen, step):
    sentences = list()
    next_chars = list()
    for i in range(0, len(text) - maxLen, step):
        sentences.append(text[i: i + maxLen])
        next_chars.append(text[i + maxLen])
    print('nb seq: ', len(sentences))
    return sentences, next_chars

maxLen = 40
step = 3
sentences, next_chars = create_seq(text, maxLen, step)
# TODO: need to use sentences rather than sequen

nb seq:  200285


In [22]:
# Vectorization
x = np.zeros( (len(sentences), maxLen, numChars), dtype=np.bool )
y = np.zeros( (len(sentences), numChars), dtype=np.bool )

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [24]:
# model: a single LSTM
model = Sequential()
model.add(LSTM(units=128, input_shape=(maxLen, numChars)))
model.add(Dense(numChars))
model.add(Activation('softmax'))